In [1]:
import pandas as pd
import networkx as nx
import numpy as np

In [2]:
df_SC = pd.read_csv('SC_summary.csv', parse_dates=['start_date', 'end_date'])
df_SC.replace({pd.NaT: np.datetime64(800, 'M')}, inplace=True)

dates = [np.datetime64(592 + 3*i, 'M') for i in range(14)]

df_customer_cats = df_SC[["customer_SIC", "SIC_desc"]].drop_duplicates().sort_values(by='customer_SIC').rename(columns={"customer_SIC": "SIC", "SIC_desc": "SIC_desc"})
df_supplier_cats = df_SC[["supplier_SIC", "SIC_desc_s"]].drop_duplicates().sort_values(by='supplier_SIC').rename(columns={"supplier_SIC": "SIC", "SIC_desc_s": "SIC_desc"})

all_sic_codes = pd.concat([df_customer_cats, df_supplier_cats]).drop_duplicates().sort_values(by='SIC')

In [5]:
def sic_code_to_sector(sic_code):
    if sic_code < 1000:
        return "Agricultural"
    if sic_code < 1500:
        return "Mining"
    if sic_code<2000:
        return "Construction"
    if sic_code<3000:
        return "Consumer goods"
    if sic_code < 3500:
        return "Basic Industrial goods"
    if sic_code < 4000:
        return "Advanced industrial goods"
    if sic_code < 5000:
        return "Infrastructure"
    if sic_code < 6000:
        return "Retail"
    if sic_code < 7000:
        return "Financial"
    if sic_code < 8000:
        return "Services"
    if sic_code < 9000:
        return "Advanced services"
    else:
        return "Government"
    
all_sectors = ["Agricultural", "Mining", "Construction", "Consumer goods", "Basic Industrial goods", "Advanced industrial goods", "Infrastructure", "Retail", "Financial","Services", "Advanced services", "Government"]

In [7]:
def get_relevant_connections_for_date(df_SC, date):
    df_SC_date = df_SC.query('start_date <= @date <= end_date')
    return df_SC_date.reset_index().drop(columns=['index'])

def convert_df_to_graph(df):
    G = nx.MultiDiGraph()
    for row_nr, row in df.iterrows():
        start_date, end_date, customer_ID, cusomter_name, customer_SIC, SIC_desc, supplier_ID, supplier_name, supplier_SIC, SIC_desc_s = row
        G.add_node(customer_ID, name=cusomter_name, SIC = customer_SIC, desc=SIC_desc, global_sec = sic_code_to_sector(customer_SIC)) # id=customer_SIC, desc=SIC_desc
        G.add_node(supplier_ID, name=supplier_name, SIC = supplier_SIC, desc=SIC_desc_s, global_sec = sic_code_to_sector(supplier_SIC)) #  id=supplier_SIC, desc=SIC_desc_s
        G.add_edge(supplier_ID, customer_ID, customer=customer_ID, customer_name = cusomter_name, supplier=supplier_ID, supplier_name=supplier_name, goods=supplier_SIC, descr=SIC_desc_s, edge_id=row_nr)
    return G

# stats on centrality, closeness etc -> see networkx


def get_suppliers(df, firmID):
    suppliers = df[df['customer_ID'] == firmID]
    if suppliers.empty:
        return suppliers, None
    
    sector_client_SIC = int(suppliers['customer_SIC'].median())
    if sector_client_SIC not in suppliers['customer_SIC'].values:
        sector_client_SIC = int(suppliers['customer_SIC'].max())
    return suppliers, sector_client_SIC

def get_clients(df, firmID):
    clients = df[df['supplier_ID'] == firmID]
    if clients.empty:
        return clients, None
    sector_supplier_SIC = int(clients['supplier_SIC'].median())
    if sector_client_SIC not in clients['supplier_SIC'].values:
        sector_client_SIC = int(clients['supplier_SIC'].max())
    return clients, sector_supplier_SIC


def score_dualsourcing(df, G):
    # scoring:
    # global model
    n= len(all_sectors)
    list_sic_codes = list(all_sic_codes['SIC'].values)
    sectors_dualsourced = pd.DataFrame(np.zeros((n,n)), columns = all_sectors, index = all_sectors)
    sectors_not_dualsourced = pd.DataFrame(np.zeros((n,n)), columns = all_sectors, index = all_sectors)
    
    total_dual_sourcers = 0
    total_firms = 0

    percentage_dualsourcing = 0
    for node in G.nodes:
        supplier_of_goods, SIC_code = get_suppliers(df, node)
        if supplier_of_goods.empty:
            continue

        suppliers_per_sector = supplier_of_goods['supplier_SIC'].value_counts()
        amnt_dualsourced_suppliers = sum(suppliers_per_sector>1)

        does_dual_sourcing = amnt_dualsourced_suppliers > 0

        if does_dual_sourcing:
            total_dual_sourcers += 1

        total_firms += 1

        sectors_that_were_dualsourced = list(suppliers_per_sector[suppliers_per_sector>1].index)
        sectors_that_were_not_dualsourced = list(suppliers_per_sector[suppliers_per_sector<=1].index)

        for supplier_sector_SIC in sectors_that_were_dualsourced:
            sectors_dualsourced[sic_code_to_sector(supplier_sector_SIC)][sic_code_to_sector(SIC_code)] += 1

        for supplier_sector_SIC in sectors_that_were_not_dualsourced:
            sectors_not_dualsourced[sic_code_to_sector(supplier_sector_SIC)][sic_code_to_sector(SIC_code)] += 1


    return sectors_dualsourced, sectors_not_dualsourced, total_dual_sourcers/total_firms


list_G = []
list_sectors_dualsourced = []
list_sectors_not_dualsourced = []
list_does_any_dualsourcing = []
for date in dates:
    df = get_relevant_connections_for_date(df_SC, date)
    #df = df.head(25)
    G = convert_df_to_graph(df)
    nx.write_graphml(G, f'001_full_network_{date}_sectorial.graphml', prettyprint=True, edge_id_from_attribute='edge_id')
    sectors_dualsourced, sectors_not_dualsourced, pct_any_dualsource = score_dualsourcing(df, G)

    list_G.append(G)
    list_sectors_dualsourced.append(sectors_dualsourced)
    list_sectors_not_dualsourced.append(sectors_not_dualsourced)
    list_does_any_dualsourcing.append(pct_any_dualsource)

In [8]:
import pickle

with open('ldualsec.pickle', 'wb') as handle:
    pickle.dump(list_sectors_dualsourced, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('lnodualsec.pickle', 'wb') as handle:
    pickle.dump(list_sectors_not_dualsourced, handle, protocol=pickle.HIGHEST_PROTOCOL)


with open('nodualsectorsm2.pickle', 'wb') as handle:
    pickle.dump(list_does_any_dualsourcing, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
list_pct_dualsourcing = [df.sum().sum() / (df.sum().sum() + df2.sum().sum()) for df, df2 in zip(list_sectors_dualsourced, list_sectors_not_dualsourced)]
abs_amnt_dual_sourcing = [df.sum().sum() for df, df2 in zip(list_sectors_dualsourced, list_sectors_not_dualsourced)]

In [22]:
import plotly.express as px

px.line(x=dates, y=list_pct_dualsourcing,
        labels={
            "x": "Date",
            "y": "Average dualsourcing per supplier sector",
        },
        title='Company-level view on suppliers: average dualsourcing per distinct supplier sector')

#(measured as two or more suppliers from the same sector)

In [21]:
fig = px.line(x=dates, y=[100*val for val in list_does_any_dualsourcing],
        labels={
            "x": "Date",
            "y": "percentage of companies utilising dualsourcing",
        },
        title='Firms that employ any kind of dual sourcing practices')

fig.update_layout(yaxis_ticksuffix = "%")
fig.show()

In [24]:
list_sectors_dualsourced[1].sum()

Agricultural                   50.0
Mining                        623.0
Construction                  370.0
Consumer goods               2496.0
Basic Industrial goods       1089.0
Advanced industrial goods    5247.0
Infrastructure               1790.0
Retail                        771.0
Financial                    1329.0
Services                     4094.0
Advanced services             735.0
Government                     14.0
dtype: float64

In [27]:
list_sectors_not_dualsourced[1].sum()

Agricultural                   318.0
Mining                        1463.0
Construction                  1643.0
Consumer goods                7888.0
Basic Industrial goods        4679.0
Advanced industrial goods    13492.0
Infrastructure                4569.0
Retail                        3749.0
Financial                     4151.0
Services                      8531.0
Advanced services             2584.0
Government                     153.0
dtype: float64

In [26]:
list_sectors_not_dualsourced[1].sum() + list_sectors_dualsourced[1].sum()

Agricultural                   368.0
Mining                        2086.0
Construction                  2013.0
Consumer goods               10384.0
Basic Industrial goods        5768.0
Advanced industrial goods    18739.0
Infrastructure                6359.0
Retail                        4520.0
Financial                     5480.0
Services                     12625.0
Advanced services             3319.0
Government                     167.0
dtype: float64

In [31]:
px.bar(list_sectors_dualsourced[0].sum() / (list_sectors_not_dualsourced[0].sum() + list_sectors_dualsourced[0].sum()))

In [32]:
list_results = [df.sum() / (df.sum() + df2.sum()) for df, df2 in zip(list_sectors_dualsourced, list_sectors_not_dualsourced)]

In [33]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [43]:
dates

[numpy.datetime64('2019-05'),
 numpy.datetime64('2019-08'),
 numpy.datetime64('2019-11'),
 numpy.datetime64('2020-02'),
 numpy.datetime64('2020-05'),
 numpy.datetime64('2020-08'),
 numpy.datetime64('2020-11'),
 numpy.datetime64('2021-02'),
 numpy.datetime64('2021-05'),
 numpy.datetime64('2021-08'),
 numpy.datetime64('2021-11'),
 numpy.datetime64('2022-02'),
 numpy.datetime64('2022-05'),
 numpy.datetime64('2022-08')]

In [49]:
fig = go.Figure()
anchos = [0.2] * 11

fig.add_trace(go.Bar(x=list_results[0].index, y=list_results[0], name=str(dates[0])))


fig.add_trace(go.Bar(x=list_results[8].index, y=list_results[8], name=str(dates[8])))

fig.add_trace(go.Bar(x=list_results[13].index, y=list_results[13], name=str(dates[13])))

fig.update_layout(title =  "Dual sourcing per supplier sector", yaxis_title="Percentage dual sourcing")
fig

In [52]:
list_results2 = [df.sum(axis=1) / (df.sum(axis=1) + df2.sum(axis=1)) for df, df2 in zip(list_sectors_dualsourced, list_sectors_not_dualsourced)]

In [53]:
fig = go.Figure()
anchos = [0.2] * 11

fig.add_trace(go.Bar(x=list_results2[0].index, y=list_results2[0], name=str(dates[0])))


fig.add_trace(go.Bar(x=list_results2[8].index, y=list_results2[8], name=str(dates[8])))

fig.add_trace(go.Bar(x=list_results2[13].index, y=list_results2[13], name=str(dates[13])))

fig.update_layout(title =  "Dual sourcing per client sector", yaxis_title="Percentage dual sourcing")
fig